In [4]:
import sys
import pathlib
import pickle
from itertools import chain

from yacs.config import CfgNode
from tqdm.notebook import tqdm
import numpy as np
import torch
import trimesh

deca_d = '/vision/changan/shrinidhi/DECA'
sys.path.append(deca_d)

from decalib.models.FLAME import FLAME

In [2]:
data_d = pathlib.Path('/vision/vision_data_2/VGGSound_shards_fixed/shrinidhi/lm_listener/dataset')

In [5]:
train_segs = [torch.load(f)
              for f in data_d.glob('*/segments_train.pth')]
train_segs = list(chain(*train_segs))
len(train_segs)

26832

In [6]:
val_segs = [torch.load(f)
              for f in data_d.glob('*/segments_val.pth')]
val_segs = list(chain(*val_segs))
len(val_segs)

2840

In [7]:
test_segs = [torch.load(f)
              for f in data_d.glob('*/segments_test.pth')]
test_segs = list(chain(*test_segs))
len(test_segs)

8534

In [8]:
for k,v in train_segs[0].items():
    if isinstance(v,torch.Tensor):
        print(k,v.shape)
    else:
        print(k,v)

start 207.942
end 267.95
speaker SPEAKER_00
split_start_frame 7236
split_end_frame 7922
split_start_time 241.2
split_end_time 264.06666666666666
fname conan_videos/done_conan_videos0/079YouTube/079YouTube.mp4
before_words [{'text': 'Well,', 'start': 208.46, 'end': 209.5, 'confidence': 0.998}, {'text': 'I', 'start': 209.5, 'end': 209.7, 'confidence': 0.987}, {'text': "don't", 'start': 209.7, 'end': 209.96, 'confidence': 0.838}, {'text': 'know.', 'start': 209.96, 'end': 210.43, 'confidence': 0.984}, {'text': 'I', 'start': 210.43, 'end': 210.47, 'confidence': 1.0}, {'text': 'think', 'start': 210.47, 'end': 210.54, 'confidence': 1.0}, {'text': 'Ted', 'start': 210.54, 'end': 210.74, 'confidence': 0.995}, {'text': 'Sarandos', 'start': 210.74, 'end': 211.38, 'confidence': 0.917}, {'text': 'who', 'start': 211.38, 'end': 211.56, 'confidence': 0.722}, {'text': 'runs', 'start': 211.56, 'end': 211.76, 'confidence': 0.999}, {'text': 'Netflix', 'start': 211.76, 'end': 212.18, 'confidence': 0.998}, {

In [9]:
model = pickle.load(open('/vision/changan/shrinidhi/DECA/data/generic_model.pkl','rb'),encoding='latin1')
for k,v in model.items():
    if hasattr(v,'shape'):
        print(k,v.shape)
    else:
        print(k,v)

f (9976, 3)
J_regressor (5, 5023)
kintree_table (2, 5)
J (5, 3)
bs_style lbs
weights (5023, 5)
posedirs (5023, 3, 36)
v_template (5023, 3)
shapedirs (5023, 3, 400)
bs_type lrotmin


/tmp/ipykernel_219956/1564281621.py:1: DeprecationWarning: Please import `csc_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csc` namespace is deprecated and will be removed in SciPy 2.0.0.
  model = pickle.load(open('/vision/changan/shrinidhi/DECA/data/generic_model.pkl','rb'),encoding='latin1')


In [10]:
cfg = CfgNode({'flame_model_path':f'{deca_d}/data/generic_model.pkl',
               'flame_lmk_embedding_path':f'{deca_d}/data/landmark_embedding.npy',
               'n_shape':100,
               'n_exp':50,
               'n_pose':6})
flame = FLAME(cfg)

creating the FLAME Decoder


In [11]:
examples = test_segs
meshes_d = pathlib.Path('/vision/vision_data_2/VGGSound_shards_fixed/shrinidhi/lm_listener/meshes')

In [12]:
for i_e,example in enumerate(tqdm(examples[:12],desc='examples')):
    ply_d = meshes_d / f'{i_e:05}' / 'p0'
    ply_d.mkdir(exist_ok=True,parents=True)
    verts,_,_ = flame(shape_params=example['p0_shape'],
                      expression_params=example['p0_exp'],
                      pose_params=example['p0_pose'])
    for i in range(verts.shape[0]):
        ply_f = f'{ply_d}/{i:05}.ply'
        mesh = trimesh.Trimesh(vertices=verts[i],
                                faces=flame.faces_tensor,
                                process=False)
        mesh.export(ply_f)
    # ply_d = meshes_d / f'{i_e:05}' / 'p1'
    # ply_d.mkdir(exist_ok=True,parents=True)
    # verts,_,_ = flame(shape_params=example['p1_shape'],
    #                   expression_params=example['p1_exp'],
    #                   pose_params=example['p1_pose'])
    # for i in range(verts.shape[0]):
    #     ply_f = f'{ply_d}/{i:05}.ply'
    #     mesh = trimesh.Trimesh(vertices=verts[i],
    #                             faces=flame.faces_tensor,
    #                             process=False)
    #     mesh.export(ply_f)


examples:   0%|          | 0/12 [00:00<?, ?it/s]